# Step 13: V10 — Aperture dependence (per sector)

Goal: test whether the measured transit depth changes systematically with aperture size.

Why this matters:
- If depth increases with larger apertures, it can indicate contamination from nearby sources.
- If depth is stable across apertures, it supports an on-target origin.

Notes:
- This check requires a per-sector TPF.
- Interpret alongside V08 (centroid shift) and V09 (difference image).


In [ ]:
from __future__ import annotations

from pathlib import Path
import json
import sys

import numpy as np

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh
import tess_vetter.api as btv

ds = sh.load_dataset()

stitched = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(stitched)
candidate = sh.make_candidate(depth_ppm)

# Load TPF stamps from the per-sector cache (created by V08).
tpf_by_sector: dict[int, btv.TPFStamp] = {int(k): v for k, v in ds.tpf_by_sector.items()}
cache_dir = Path('persistent_cache/tutorial_toi-5807-incremental/tpfs')
sectors = sorted(int(s) for s in ds.lc_by_sector.keys())

for sector in sectors:
    if sector in tpf_by_sector:
        continue
    npz = cache_dir / f'sector{sector}_tpf.npz'
    d = np.load(npz, allow_pickle=True)
    try:
        from astropy.wcs import WCS

        wcs = WCS(d['wcs_header'].item()) if 'wcs_header' in d else None
    except Exception:
        wcs = d['wcs_header'].item() if 'wcs_header' in d else None

    tpf_by_sector[sector] = btv.TPFStamp(
        time=np.asarray(d['time'], dtype=np.float64),
        flux=np.asarray(d['flux'], dtype=np.float64),
        flux_err=np.asarray(d['flux_err'], dtype=np.float64) if 'flux_err' in d else None,
        wcs=wcs,
        aperture_mask=np.asarray(d['aperture_mask'], dtype=bool) if 'aperture_mask' in d else None,
        quality=np.asarray(d['quality'], dtype=np.int32) if 'quality' in d else None,
    )

# Run V10 per sector (compact summary)
by_sector = {}
for sector in sectors:
    lc0 = ds.lc_by_sector[sector]
    q = np.asarray(
        lc0.quality if getattr(lc0, 'quality', None) is not None else np.zeros(len(lc0.time)),
        dtype=np.int32,
    )
    lc_sec = btv.LightCurve(time=lc0.time, flux=lc0.flux, flux_err=lc0.flux_err, quality=q)
    tpf = tpf_by_sector.get(sector)
    if tpf is None:
        by_sector[str(sector)] = {'status': 'skipped', 'flags': ['MISSING_TPF'], 'metrics': {}}
        continue

    session = btv.VettingSession.from_api(
        lc=lc_sec,
        candidate=candidate,
        stellar=sh.STELLAR,
        tpf=tpf,
        network=False,
        tic_id=sh.TIC_ID,
    )
    r = session.run('V10')
    m = dict(r.metrics)
    by_sector[str(sector)] = {
        'status': r.status,
        'flags': r.flags,
        'metrics': {
            'depth_ppm_aperture_min': m.get('depth_ppm_aperture_min'),
            'depth_ppm_aperture_max': m.get('depth_ppm_aperture_max'),
            'depth_variance_ppm2': m.get('depth_variance_ppm2'),
            'stability_metric': m.get('stability_metric'),
            'recommended_aperture_pixels': m.get('recommended_aperture_pixels'),
            'aperture_depth_sign_flip': m.get('aperture_depth_sign_flip'),
            'drift_fraction_recommended': m.get('drift_fraction_recommended'),
            'n_transit_epochs': m.get('n_transit_epochs'),
        },
    }

print(json.dumps(by_sector, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output (per-sector summary)</b></summary>

```text
{
  "55": {
    "flags": [],
    "metrics": {
      "aperture_depth_sign_flip": false,
      "depth_ppm_aperture_max": 200.5432275224983,
      "depth_ppm_aperture_min": 199.07785479117422,
      "depth_variance_ppm2": 61.93261645250078,
      "drift_fraction_recommended": 0.0027481705590840983,
      "n_transit_epochs": 2,
      "recommended_aperture_pixels": 2.0,
      "stability_metric": 0.9476333172012621
    },
    "status": "ok"
  },
  "75": {
    "flags": [
      "negative_depths_present"
    ],
    "metrics": {
      "aperture_depth_sign_flip": true,
      "depth_ppm_aperture_max": 244.16930738530016,
      "depth_ppm_aperture_min": -8.56705518570422,
      "depth_variance_ppm2": 8698.152366206981,
      "drift_fraction_recommended": -0.0027741598276521335,
      "n_transit_epochs": 2,
      "recommended_aperture_pixels": 2.0,
      "stability_metric": 0.4001345694379497
    },
    "status": "ok"
  },
  "82": {
    "flags": [],
    "metrics": {
      "aperture_depth_sign_flip": false,
      "depth_ppm_aperture_max": 154.63275974902936,
      "depth_ppm_aperture_min": 335.9793246395237,
      "depth_variance_ppm2": 4437.305770408408,
      "drift_fraction_recommended": 0.00592863778349,
      "n_transit_epochs": 2,
      "recommended_aperture_pixels": 1.5,
      "stability_metric": 0.6371928139759866
    },
    "status": "ok"
  },
  "83": {
    "flags": [],
    "metrics": {
      "aperture_depth_sign_flip": false,
      "depth_ppm_aperture_max": 249.4213885769203,
      "depth_ppm_aperture_min": 240.0796456475951,
      "depth_variance_ppm2": 162.71352257120816,
      "drift_fraction_recommended": -0.001826771688093012,
      "n_transit_epochs": 2,
      "recommended_aperture_pixels": 2.0,
      "stability_metric": 0.9280707188941079
    },
    "status": "ok"
  }
}
```

</details>


In [ ]:
# Plot V10: Aperture dependence (single sector example)
out = {}

try:
    import matplotlib.pyplot as plt
    from tess_vetter.api import plot_aperture_curve
    PLOTTING_AVAILABLE = True
except Exception as e:
    PLOTTING_AVAILABLE = False
    out['plotting_error'] = str(e)

if PLOTTING_AVAILABLE:
    sector = max(int(s) for s in ds.lc_by_sector.keys())
    lc0 = ds.lc_by_sector[sector]
    q = np.asarray(
        lc0.quality if getattr(lc0, 'quality', None) is not None else np.zeros(len(lc0.time)),
        dtype=np.int32,
    )
    lc_sec = btv.LightCurve(time=lc0.time, flux=lc0.flux, flux_err=lc0.flux_err, quality=q)
    tpf = tpf_by_sector.get(sector)
    if tpf is None:
        raise RuntimeError(f'Missing TPF for sector {sector}')

    session = btv.VettingSession.from_api(
        lc=lc_sec,
        candidate=candidate,
        stellar=sh.STELLAR,
        tpf=tpf,
        network=False,
        tic_id=sh.TIC_ID,
    )
    r = session.run('V10')

    run_out_dir, docs_out_dir = sh.artifact_dirs(step_id='13_v10_aperture_dependence')
    run_path = run_out_dir / 'V10_aperture_curve.png'
    docs_path = (docs_out_dir / 'V10_aperture_curve.png') if docs_out_dir is not None else None

    fig, ax = plt.subplots(figsize=(8, 5))
    plot_aperture_curve(r, ax=ax)
    ax.set_title(f'V10: Aperture dependence (sector {sector})')
    fig.tight_layout()
    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()

    out['sector_plotted'] = int(sector)
    out['flags'] = r.flags
    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


**Pre-rendered plot (no execution required):** `../artifacts/tutorial_toi-5807-incremental/13_v10_aperture_dependence/V10_aperture_curve.png`

![V10: Aperture dependence](../artifacts/tutorial_toi-5807-incremental/13_v10_aperture_dependence/V10_aperture_curve.png)


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/13_v10_aperture_dependence/V10_aperture_curve.png",
  "flags": [],
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/13_v10_aperture_dependence/V10_aperture_curve.png",
  "sector_plotted": 83
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Flags:** sector-dependent (e.g. sector 75 has `negative_depths_present` and a sign flip).
- **Result:** sector 83 is stable across apertures (∼240–249 ppm across radii 1–3 px).
- **Cross-sector note:** some sectors show instability/sign flips (e.g. sector 75 has `negative_depths_present` and a sign flip).
- **Why it’s useful:** aperture dependence helps separate on-target signals from blends and also reveals sector-specific systematics.
- **Interpretation:** evidence is mixed across sectors; treat sector 83 as supportive, and treat unstable sectors as lower-confidence for pixel-based conclusions.
- **Next step:** proceed to exovetter-style uniqueness/variability checks (V11 ModShift, V12 SWEET).

</details>
